# Raising Exceptions — Practice (Advanced, but not too much)

Best practices we’ll practice here:

- Raise **specific** exceptions (e.g., `ValueError`, `TypeError`, `KeyError`, custom exceptions).
- Provide **clear, actionable messages**.
- Use **exception chaining** (`raise ... from err`) when adding context.
- Prefer **EAFP** style when appropriate; convert low-level errors to domain-specific ones.
- When validating multiple fields, consider `ExceptionGroup` (Python 3.11+).

Each task includes a self-check cell with assertions. Run it to verify your solution.

### Test Helpers (for asserts)
Tiny utilities to assert that a function raises a particular exception and optionally check the message and chaining.

In [1]:
def assert_raises(exc_type, func, *args, **kwargs):
    try:
        func(*args, **kwargs)
    except exc_type as e:
        return e  # return the exception for further inspection
    except Exception as e:  # wrong exception
        raise AssertionError(f"Expected {exc_type.__name__}, got {type(e).__name__}: {e}")
    else:
        raise AssertionError(f"Expected {exc_type.__name__} to be raised, but no exception occurred.")

def assert_message_contains(exc, text):
    assert text in str(exc), f"Message does not contain {text!r}: {exc}"

## 1) Validate Username (Type + Length)
Write `validate_username(name)` that:
- Raises `TypeError('name must be a str')` if `name` is not a string.
- Strips whitespace; raises `ValueError('name length must be between 5 and 20')` if length not in `[5, 20]`.
- Returns the cleaned name otherwise.

In [2]:
# TODO: implement validate_username as specified
def validate_username(name):
    if not isinstance(name, str):
        raise TypeError('name must be a str')
    cleaned = name.strip()
    if not (5 <= len(cleaned) <= 20):
        raise ValueError('name length must be between 5 and 20')
    return cleaned

# quick demo
validate_username('  alice  ')

'alice'

In [3]:
_e = assert_raises(TypeError, validate_username, 123)
assert_message_contains(_e, 'name must be a str')
_e = assert_raises(ValueError, validate_username, ' ed ')
assert_message_contains(_e, 'between 5 and 20')
assert validate_username('  verygoodname  ') == 'verygoodname'
print('OK - 1')

OK - 1


## 2) Safe Divide with Explicit Zero Check
Write `safe_divide(a, b)` that:
- Raises `TypeError('a and b must be numbers')` if `a` or `b` is not `int`/`float`.
- Raises `ZeroDivisionError('denominator cannot be zero')` if `b == 0`.
- Returns `a / b` otherwise.

In [4]:
# TODO: implement safe_divide
def safe_divide(a, b):
    if not isinstance(a, (int, float)) or not isinstance(b, (int, float)):
        raise TypeError('a and b must be numbers')
    if b == 0:
        raise ZeroDivisionError('denominator cannot be zero')
    return a / b

safe_divide(10, 2)

5.0

In [5]:
_e = assert_raises(TypeError, safe_divide, '10', 2)
assert_message_contains(_e, 'numbers')
_e = assert_raises(ZeroDivisionError, safe_divide, 10, 0)
assert_message_contains(_e, 'denominator')
assert abs(safe_divide(9, 4) - 2.25) < 1e-12
print('OK - 2')

OK - 2


## 3) Median of a **Sorted** Sequence
Write `median_strict(seq)` that:
- Raises `TypeError('seq must be an iterable of numbers')` if elements aren’t numbers.
- Raises `ValueError('seq must be non-empty')` if empty.
- Raises `ValueError('seq must be sorted ascending')` if `seq` is not sorted non-decreasing.
- Returns the median otherwise (for even length, average the two middles).

In [6]:
# TODO: implement median_strict
def median_strict(seq):
    data = list(seq)
    if not data:
        raise ValueError('seq must be non-empty')
    for x in data:
        if not isinstance(x, (int, float)):
            raise TypeError('seq must be an iterable of numbers')
    if any(data[i] > data[i+1] for i in range(len(data)-1)):
        raise ValueError('seq must be sorted ascending')
    n = len(data)
    mid = n // 2
    if n % 2:
        return data[mid]
    return (data[mid-1] + data[mid]) / 2

median_strict([1, 2, 3, 4])

2.5

In [7]:
_e = assert_raises(ValueError, median_strict, [])
assert_message_contains(_e, 'non-empty')
_e = assert_raises(TypeError, median_strict, [1, 'x'])
assert_message_contains(_e, 'numbers')
_e = assert_raises(ValueError, median_strict, [1, 3, 2])
assert_message_contains(_e, 'sorted ascending')
assert median_strict([1, 2, 3]) == 2
assert median_strict([1, 2, 3, 4]) == 2.5
print('OK - 3')

OK - 3


## 4) Nested Mapping Lookup with Clear KeyError
Write `get_nested(mapping, keys, default=_MISSING)` that descends a mapping by `keys` (an iterable of keys):
- If any key is missing and `default` is **not** provided, raise `KeyError` with a **dotted path** (e.g., `'config.db.port'`).
- If `default` **is** provided, return it when a key is missing.
- Otherwise, return the found value.

> Focus on raising a **useful KeyError message**.

In [8]:
# TODO: implement get_nested
_MISSING = object()

def get_nested(mapping, keys, default=_MISSING):
    cur = mapping
    path = []
    for k in keys:
        path.append(str(k))
        try:
            cur = cur[k]
        except Exception as err:
            if default is _MISSING:
                dotted = '.'.join(path)
                raise KeyError(dotted) from err
            return default
    return cur

cfg = {'config': {'db': {'port': 5432}}}
get_nested(cfg, ['config', 'db', 'port'])

5432

In [9]:
assert get_nested(cfg, ['config', 'db', 'port']) == 5432
assert get_nested(cfg, ['config', 'cache'], default='none') == 'none'
_e = assert_raises(KeyError, get_nested, cfg, ['config', 'missing'])
assert str(_e) in ("'config.missing'", 'config.missing')  # KeyError may repr key
assert isinstance(_e.__cause__, (KeyError, TypeError))
print('OK - 4')

OK - 4


## 5) Parsing with Exception Chaining
Write `parse_int(s)` that returns an integer:
- Accepts strings with optional whitespace and leading `+/-`.
- On failure (bad type or bad content) **raise** `ValueError(f"cannot parse int from {s!r}")` and chain the original error using `from`.
- Demonstrate that `__cause__` is set.

In [10]:
# TODO: implement parse_int with chaining
def parse_int(s):
    try:
        return int(s.strip())
    except Exception as err:
        raise ValueError(f"cannot parse int from {s!r}") from err

parse_int('  +42 ')


42

In [11]:
assert parse_int(' -7 ') == -7
_e = assert_raises(ValueError, parse_int, 'xyz')
assert_message_contains(_e, 'cannot parse int')
assert _e.__cause__ is not None
print('OK - 5')

OK - 5


## 6) Validate Config with `ExceptionGroup`
Write `validate_config(d)` that checks keys: `host` (non-empty `str`), `port` (`int` in `[1, 65535]`), `debug` (`bool`).
- Collect **all** validation issues and raise an `ExceptionGroup('config errors', [ ... ])` with one `ValueError` per problem.
- If there are no issues, return the cleaned dict (`host`, `port`, `debug`).

In [12]:
# TODO: implement validate_config using ExceptionGroup
def validate_config(d):
    errs = []
    host = d.get('host')
    port = d.get('port')
    debug = d.get('debug')

    if not isinstance(host, str) or not host.strip():
        errs.append(ValueError('host must be a non-empty string'))
    if not isinstance(port, int) or not (1 <= port <= 65535):
        errs.append(ValueError('port must be an int in [1, 65535]'))
    if not isinstance(debug, bool):
        errs.append(ValueError('debug must be a bool'))

    if errs:
        raise ExceptionGroup('config errors', errs)
    return {'host': host.strip(), 'port': port, 'debug': debug}

validate_config({'host': 'example.com', 'port': 8080, 'debug': True})

{'host': 'example.com', 'port': 8080, 'debug': True}

In [13]:
ok = validate_config({'host': ' ex ', 'port': 80, 'debug': False})
assert ok == {'host': 'ex', 'port': 80, 'debug': False}
try:
    validate_config({'host': '', 'port': 70000, 'debug': 'no'})
except* ValueError as eg:  # PEP 654 syntax; matches the grouped ValueErrors
    # We expect 3 distinct ValueErrors inside the group
    errs = eg.exceptions
    msgs = sorted(str(e) for e in errs)
    assert msgs == [
        'debug must be a bool',
        'host must be a non-empty string',
        'port must be an int in [1, 65535]'
    ]
else:
    raise AssertionError('Expected ExceptionGroup of ValueError to be raised')
print('OK - 6')

OK - 6


## 7) Custom Exception for Out-of-Range Values
Define `class RangeError(Exception)` that stores `value`, `low`, and `high` and prints a friendly message.
Implement `clamp(value, low, high)` that:
- Raises `ValueError('low must be ≤ high')` if bounds are invalid.
- Raises `RangeError` if `value` is not within `[low, high]`.
- Returns `value` otherwise (no clamping; just validation here).

In [14]:
# TODO: implement RangeError and clamp
class RangeError(Exception):
    def __init__(self, value, low, high):
        self.value = value
        self.low = low
        self.high = high
        super().__init__(f'value {value!r} is not in [{low!r}, {high!r}]')

def clamp(value, low, high):
    if low > high:
        raise ValueError('low must be ≤ high')
    if not (low <= value <= high):
        raise RangeError(value, low, high)
    return value

clamp(5, 0, 10)

5

In [15]:
_e = assert_raises(ValueError, clamp, 1, 5, 2)
assert_message_contains(_e, 'low must be')
_e = assert_raises(RangeError, clamp, -1, 0, 3)
assert_message_contains(_e, 'not in [')
assert clamp(3, 0, 3) == 3
print('OK - 7')

OK - 7


## 8) Add Context via a Context Manager (Chaining)
Create a context manager `add_context(label)` that, if an exception occurs inside the `with` block, re-raises it as `RuntimeError(f"{label}: {orig}")` **chained** from the original.

Implement `area_of_ring(outer, inner)` which computes `π(outer² - inner²)` but:
- Raises `ValueError('radii must be non-negative')` if either radius is negative.
- Raises `ValueError('outer must be > inner')` if not strictly larger.
- Wraps its logic in `with add_context('area_of_ring'):` to demonstrate chaining.

Use `math.pi` for π.

In [16]:
# TODO: implement add_context and area_of_ring with chaining
import math
from contextlib import AbstractContextManager

class add_context(AbstractContextManager):
    def __init__(self, label):
        self.label = label
    def __exit__(self, exc_type, exc, tb):
        if exc is None:
            return False  # no exception
        raise RuntimeError(f"{self.label}: {exc}") from exc

def area_of_ring(outer, inner):
    with add_context('area_of_ring'):
        if outer < 0 or inner < 0:
            raise ValueError('radii must be non-negative')
        if outer <= inner:
            raise ValueError('outer must be > inner')
        return math.pi * (outer*outer - inner*inner)

area_of_ring(3, 2)

15.707963267948966

In [17]:
_e = assert_raises(RuntimeError, area_of_ring, -1, 0)
assert_message_contains(_e, 'area_of_ring: radii must be non-negative')
assert _e.__cause__ is not None and isinstance(_e.__cause__, ValueError)
_e = assert_raises(RuntimeError, area_of_ring, 2, 2)
assert_message_contains(_e, 'outer must be > inner')
assert abs(area_of_ring(5, 3) - (math.pi * (25 - 9))) < 1e-12
print('OK - 8')

OK - 8


## 9) Environment Parsing (EAFP + Conversion)
Write `get_port(env)` that reads port from mapping `env['PORT']` and returns it as an `int`.
- On missing key or non-integer value, raise **one** `ValueError('PORT must be an integer env var')` and chain the original (`from e`).

In [18]:
# TODO: implement get_port using EAFP and chaining
def get_port(env):
    try:
        return int(env['PORT'])
    except Exception as e:
        raise ValueError('PORT must be an integer env var') from e

get_port({'PORT': '8080'})

8080

In [19]:
assert get_port({'PORT': '5000'}) == 5000
_e = assert_raises(ValueError, get_port, {})
assert_message_contains(_e, 'PORT must be an integer env var')
assert _e.__cause__ is not None  # KeyError chained
_e = assert_raises(ValueError, get_port, {'PORT': 'x'})
assert isinstance(_e.__cause__, ValueError)
print('OK - 9')

OK - 9


## 10) Re-raise After "Logging" (Don’t Swallow)
Implement `divide_and_log(a, b, log)` that:
- On `ZeroDivisionError`, appends a short message to `log` (a list) and **re-raises the original** exception (using bare `raise`).
- Otherwise returns the division result.

> Pattern: handle/annotate locally, then re-raise for callers to see.

In [20]:
# TODO: implement divide_and_log
def divide_and_log(a, b, log):
    try:
        return a / b
    except ZeroDivisionError:
        log.append(f'ZeroDivisionError for inputs a={a}, b={b}')
        raise  # re-raise the same exception

divide_and_log(6, 3, [])

2.0

In [21]:
logs = []
assert divide_and_log(8, 2, logs) == 4
try:
    divide_and_log(1, 0, logs)
except ZeroDivisionError:
    pass
else:
    raise AssertionError('Expected ZeroDivisionError to be re-raised')
assert logs and 'ZeroDivisionError' in logs[-1]
print('OK - 10')

OK - 10
